In [1]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Run TTS
# Text to speech to a file
tts.tts_to_file(text="Ei! Tudo bem com você?", speaker_wav="scarlett.wav", language="pt", file_path="output.wav")

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.


c:\Users\Andrey\Desktop\VoiceAssist\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 > Using model: xtts
 > Text splitted to sentences.
['Ei!', 'Tudo bem com você?']
 > Processing time: 22.921683073043823
 > Real-time factor: 5.908893468955952


'output.wav'

In [9]:
import json
import trafilatura
import google.generativeai as genai
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Configurações do Gemini
generation_config = {
  "temperature": 0.8,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 1024,
  "response_mime_type": "application/json",
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE",
  },
]
model = genai.GenerativeModel(
          model_name="gemini-1.5-flash-latest",
          safety_settings=safety_settings,
          generation_config=generation_config,
        )

def say(text, metadata):
    """
    Gera áudio de fala a partir do texto fornecido e salva em um arquivo WAV.

    Args:
        text (str): O texto a ser convertido em fala.
        speed (float, optional): A velocidade de fala, entre 0.1 e 2.0. Defaults to 0.9.
        filename (str, optional): O nome do arquivo WAV de saída. Defaults to "output.wav".
    """
    tts.tts_to_file(text=text, speaker_wav="scarlett.wav", language="pt", file_path=f"{metadata['title']}.wav")


def extract_metada(text):
    prompt = f"""Observe the text and extract the following information:
{'{'}
    "title": string, # Generate or Extract a title for the text
    "description": string # Short description of the content of the text
{'}'}

Text: 
```
{text}
```
"""
    done = False
    while not done:
        try:
            response = model.generate_content(prompt)
        except Exception as e:
            print(e)
    return json.loads(response)

def read_page(url):
    """
    Lê uma página da Web, extrai o texto e o converte em fala, salvando em um único arquivo WAV.

    Args:
        url (str): O URL da página da Web.
        filename (str, optional): O nome do arquivo WAV de saída. Defaults to "output.wav".
    """
    page = trafilatura.fetch_url(url)
    text = trafilatura.extract(page)
    #metadata = extract_metada(text)
    say(text, metadata={'title':'SCP 724'})

In [10]:
read_page("https://scp-wiki.wikidot.com/scp-724")

 > Text splitted to sentences.
['Item #: SCP-724', 'Object Class: Safe', 'Special Containment Procedures: Due to its immobility, SCP-724 is located in the central vault of Site-54.', 'This vault is surrounded by a set of hinged shielding plates that can be closed to block radio signals from reaching SCP-724 and suppress the anomaly.', 'SCP-724-1 specimens can be kept in standard-size cages, although any that are currently being monitored should be separated from each other by at least moderate soundproofing.', 'Individuals not being monitored are kept in kennel sections with Faraday shielding.', 'Transport of SCP-724-1 specimens must not use unshielded cages except by written permission of Researcher M███████.', 'Site-54 maintains the outward appearance of a corporate office/research facility, in the name of a Foundation front company (S██████ C██ P███████, Inc.).', 'The lobby is to be staffed by Foundation security personnel posing as civilian receptionists and security guards.', 'Des